In [1]:
#Use conda env open2c_env.yml to create conda env for this script
#This is an example script of calculating the intermingling metric (IM) for real and permuted walks
#It is run from within the 'scripts' subdirectory, using following directory structure:
#Analysis_Dir
#├── data
#    ├── permutations
#├── alignments
#├── figures
#├── scripts
#├── lsf_jobs

In [1]:
import bioframe
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpecFromSubplotSpec
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
import random
import seaborn as sns
import scipy
import pickle
from numpy import diff

from pandas import read_csv
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

%matplotlib inline

In [2]:
conditions = [
    'Ctrl_R1',
    'IAA_R1',
    'ICRF_R1',
    'IAA_ICRF_R1',
    'Ctrl_R2',
    'IAA_R2',
    'ICRF_R2',
    'IAA_ICRF_R2',
    'Ctrl_R3',
    'IAA_R3', 
    'ICRF_R3',
    'IAA_ICRF_R3'
]

long_names = {
    'Ctrl_R1' : 'TI-MC3C-Ctrl-R1-4-40',
    'IAA_R1' : 'TI-MC3C-IAA-R1-4-40',
    'ICRF_R1' : 'TI-MC3C-ICRF-R1-4-40',
    'IAA_ICRF_R1' : 'TI-MC3C-IAA-ICRF-R1-4-40',
    'Ctrl_R2' : 'TI-MC3C-Dpn-Ctrl-R3-5-06',
    'IAA_R2' : 'TI-MC3C-Dpn-IAA-R3-5-06',
    'ICRF_R2' : 'TI-MC3C-Dpn-ICRF-R3-5-06',
    'IAA_ICRF_R2' : 'TI-MC3C-Dpn-IAA-ICRF-R3-5-06',
    'Ctrl_R3' : 'TI-MC3C-Dpn-Ctrl-R4-5-13',
    'IAA_R3' : 'TI-MC3C-Dpn-IAA-R4-5-13', 
    'ICRF_R3' : 'TI-MC3C-Dpn-ICRF-R4-5-13',
    'IAA_ICRF_R3' : 'TI-MC3C-Dpn-IAA-ICRF-R4-5-13'
}


In [3]:
sampleColors = {
    'Ctrl_R1' : '#17BECF',
    'IAA_R1' : '#574D68',
    'ICRF_R1' : '#D62728',
    'IAA_ICRF_R1' : '#C6A15B',
    'Ctrl_R2' : '#17BECF',
    'IAA_R2' : '#574D68',
    'ICRF_R2' : '#D62728',
    'IAA_ICRF_R2' : '#C6A15B',
    'Ctrl_R3' : '#17BECF',
    'IAA_R3' : '#574D68',
    'ICRF_R3' : '#D62728',
    'IAA_ICRF_R3' : '#C6A15B'
}
    
sampleLineStyles = {
    'Ctrl_R1' : '-',
    'IAA_R1' : '-',
    'ICRF_R1' : '-',
    'IAA_ICRF_R1' : '-',
    'Ctrl_R2' : '--',
    'IAA_R2' : '--',
    'ICRF_R2' : '--',
    'IAA_ICRF_R2' : '--',
    'Ctrl_R3' : ':',
    'IAA_R3' : ':',
    'ICRF_R3' : ':',
    'IAA_ICRF_R3' : ':'
}

samplePlotNames = {
    'Ctrl_R1' : 'Ctrl_R1',
    'IAA_R1' : 'IAA_R1',
    'ICRF_R1' : 'ICRF_R1',
    'IAA_ICRF_R1' : 'IAA_ICRF_R1',
    'Ctrl_R2' : 'Ctrl_R2',
    'IAA_R2' : 'IAA_R2',
    'ICRF_R2' : 'ICRF_R2',
    'IAA_ICRF_R2' : 'IAA_ICRF_R2',
    'Ctrl_R3' : 'Ctrl_R3',
    'IAA_R3' : 'IAA_R3',
    'ICRF_R3' : 'ICRF_R3',
    'IAA_ICRF_R3' : 'IAA_ICRF_R3'
}

In [4]:
ctrlconds = [
    'Ctrl_R1',
    'Ctrl_R1',
    'Ctrl_R1',
    'Ctrl_R2',
    'Ctrl_R2',
    'Ctrl_R2',
    'Ctrl_R3',
    'Ctrl_R3',
    'Ctrl_R3'
]

treatconds = [
    'IAA_R1',
    'ICRF_R1',
    'IAA_ICRF_R1',
    'IAA_R2',
    'ICRF_R2',
    'IAA_ICRF_R2',
    'IAA_R3',
    'ICRF_R3',
    'IAA_ICRF_R3'
]

In [5]:
repdict = {
    'Ctrl_R1' : 'R1',
    'IAA_R1' : 'R1',
    'ICRF_R1' : 'R1',
    'IAA_ICRF_R1' : 'R1',
    'Ctrl_R2' : 'R2',
    'IAA_R2' : 'R2',
    'ICRF_R2' : 'R2',    
    'IAA_ICRF_R2' : 'R2',
    'Ctrl_R3' : 'R3',
    'IAA_R3' : 'R3',
    'ICRF_R3' : 'R3',    
    'IAA_ICRF_R3' : 'R3'
}

labeldict = {
    'Ctrl_R1' : 'Control',
    'IAA_R1' : 'IAA',
    'ICRF_R1' : 'ICRF-193',
    'IAA_ICRF_R1' : 'IAA + ICRF-193',
    'Ctrl_R2' : 'Control',
    'IAA_R2': 'IAA',
    'ICRF_R2' : 'ICRF-193',  
    'IAA_ICRF_R2' : 'IAA + ICRF-193',
    'Ctrl_R3' : 'Control',
    'IAA_R3': 'IAA',
    'ICRF_R3' : 'ICRF-193',  
    'IAA_ICRF_R3' : 'IAA + ICRF-193'
}

labelPlotColors = {
    'Control' : '#17BECF',
    'IAA' : '#574D68',
    'ICRF-193' : '#D62728',
    'IAA + ICRF-193' : '#C6A15B',
}


In [6]:
#For each of the above, can separate by A vs B compartments, or by number of chromosomes visited
comp_types = ['A', 'B', 'AB']

In [7]:
outDataDir = '..'

In [8]:
#Read in pickled files

f = open(f'{outDataDir}/data/Rad21ICRF_R1R2R3_real_walks_with_summary_firstx_length_fractions_dict_100kbEigs.pkl', 'rb')
real_walks_with_summary_firstx = pickle.load(f)
f.close()

f = open(f'{outDataDir}/data/Rad21ICRF_R1R2R3_real_walks_filtered_summarized_firstx_length_dict_100kbEigs.pkl', 'rb')
real_walks_summarized_firstx = pickle.load(f)
f.close()

In [9]:
#Calculate intermingling metric for real walks, first 6 steps, with QC filters

In [10]:
#Parameter sweep of window size, keep 1Mb window size step size for all. 

In [11]:
#Plotting entanglement for largest step - sliding window analysis

In [12]:
Intermingling_Sliding_Window_Sweep = {}

start_dist = 0 #start of first window
end_dist = 5e7 #end of last window

for window_size in [4e6, 6e6, 8e6, 1e7, 1.2e7, 1.4e7, 1.6e7]:
    window_step = 1e6
    
    Intermingling_Sliding_Window = pd.DataFrame()

    #set up windows
    for i, start_bp in enumerate(range(int(start_dist), int(end_dist-window_size), int(window_step))):
        end_bp = start_bp + int(window_size)

        #iterate through conditions for each window, all compartments
        for cond in conditions:
            df = real_walks_with_summary_firstx[cond]['length_6']
            df['Query_Fragment_Length'] = df['Query_End'] - df['Query_Start']
            grouped_walks = df.groupby('Query_Name')
            walks_min_mapq = grouped_walks.agg({'Mapping_Quality' : 'min'})
            good_walks_mapq = walks_min_mapq[walks_min_mapq['Mapping_Quality'] > 59] #use this to filter for mapq
            walks_frac_map = grouped_walks.agg({'Match_Length' : 'sum',
                                        'Query_Fragment_Length' : 'sum',
                                        'Alignment_Length' : 'sum'
                                       })
            walks_high_frac_map = walks_frac_map[
                (walks_frac_map['Match_Length']/walks_frac_map['Query_Fragment_Length']) > 0.8] #use this to filter for fraction mapped

            df2 = real_walks_summarized_firstx[cond]['length_6'].copy()
            df_cond = df2[
                (df2['Chrom_Number'] == 1) &
                (df2['Walk_Comp_Type'].isin(['A', 'B', 'AB'])) &
                (df2['Query_Name'].isin(good_walks_mapq.index)) &
                (df2['Query_Name'].isin(walks_high_frac_map.index)) &
                (df2['Near_Largest_Step_Either_Side_Frag_Num'] == 6) &
                (df2['Largest_Step'] >= start_bp) &
                (df2['Largest_Step'] < end_bp) &
                (df2['Max_NearOneLargestStepEnd_FracOfFragments'] == 5/6)
            ][['Inter_Largest_Step_Side_Step_Num']]
            df_cond['Condition'] = cond
            df_cond['Label'] = labeldict[cond]
            df_cond['Window_Midpoint'] = (start_bp + end_bp)//2
            df_cond['Walk_Comp_Type'] = 'All'
            df_cond['Replicate'] = repdict[cond]

            #add together into one dataframe
            Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)

            for comp in comp_types:
                #iterate through comp types
                df_comp = df2[
                    (df2['Chrom_Number'] == 1) &
                    (df2['Walk_Comp_Type'] == comp) &
                    (df2['Query_Name'].isin(good_walks_mapq.index)) &
                    (df2['Query_Name'].isin(walks_high_frac_map.index)) &
                    (df2['Near_Largest_Step_Either_Side_Frag_Num'] == 6) &
                    (df2['Largest_Step'] >= start_bp) &
                    (df2['Largest_Step'] < end_bp) &
                    (df2['Max_NearOneLargestStepEnd_FracOfFragments'] == 5/6)
                ][['Inter_Largest_Step_Side_Step_Num']]
                df_comp['Condition'] = cond
                df_comp['Label'] = labeldict[cond]
                df_comp['Window_Midpoint'] = (start_bp + end_bp)//2
                df_comp['Walk_Comp_Type'] = comp
                df_comp['Replicate'] = repdict[cond]

                #add together into one dataframe
                Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
            
    Intermingling_Sliding_Window_Sweep[f'{window_size}'] = Intermingling_Sliding_Window

/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

...

/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

/tmp/ipykernel_2918989/1432558523.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
/tmp/ipykernel_2918989/1432558523.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Inter

In [13]:
#save intermingling results as pickle file for later use since takes a long time to run
f = open(f'{outDataDir}/data/Rad21ICRF_R1R2R3_InterminglingSlideingWindowSweep.pkl', 'wb')
pickle.dump(Intermingling_Sliding_Window_Sweep, f)
f.close()

In [ ]:
#permutations - calculate intermingling metric

In [ ]:
#submit permutations for sliding window sweep intermingling analysis

In [14]:
iterations = range(100)
window_sizes = [4e6, 6e6, 8e6, 1e7, 1.2e7, 1.4e7, 1.6e7]

In [15]:
for i in iterations:
    for window in window_sizes:
        !bsub -q short -W 00:20 -e ../lsf_jobs/LSB_%J.err -o ../lsf_jobs/LSB_%J.log \
            -n 1 -R span[hosts=1] -R select[ib] -R rusage[mem=3000] -R select[rh=8] \
            "python Rad21ICRF_Intermingling_Permutations.py $i $outDataDir $window"

Job <8240662> is submitted to queue <short>.
Job <8240663> is submitted to queue <short>.
Job <8240664> is submitted to queue <short>.
Job <8240665> is submitted to queue <short>.
Job <8240666> is submitted to queue <short>.
Job <8240667> is submitted to queue <short>.
Job <8240668> is submitted to queue <short>.
Job <8240669> is submitted to queue <short>.
Job <8240670> is submitted to queue <short>.
Job <8240671> is submitted to queue <short>.
Job <8240672> is submitted to queue <short>.
Job <8240673> is submitted to queue <short>.
Job <8240674> is submitted to queue <short>.
Job <8240675> is submitted to queue <short>.
Job <8240676> is submitted to queue <short>.
Job <8240677> is submitted to queue <short>.
Job <8240678> is submitted to queue <short>.
Job <8240679> is submitted to queue <short>.
Job <8240680> is submitted to queue <short>.
Job <8240681> is submitted to queue <short>.
Job <8240682> is submitted to queue <short>.
Job <8240683> is submitted to queue <short>.
Job <82406

Job <8240846> is submitted to queue <short>.
Job <8240847> is submitted to queue <short>.
Job <8240848> is submitted to queue <short>.
Job <8240849> is submitted to queue <short>.
Job <8240850> is submitted to queue <short>.
Job <8240851> is submitted to queue <short>.
Job <8240852> is submitted to queue <short>.
Job <8240853> is submitted to queue <short>.
Job <8240854> is submitted to queue <short>.
Job <8240855> is submitted to queue <short>.
Job <8240856> is submitted to queue <short>.
Job <8240857> is submitted to queue <short>.
Job <8240858> is submitted to queue <short>.
Job <8240859> is submitted to queue <short>.
Job <8240860> is submitted to queue <short>.
Job <8240861> is submitted to queue <short>.
Job <8240862> is submitted to queue <short>.
Job <8240863> is submitted to queue <short>.
Job <8240864> is submitted to queue <short>.
Job <8240865> is submitted to queue <short>.
Job <8240867> is submitted to queue <short>.
Job <8240868> is submitted to queue <short>.
Job <82408

Job <8241112> is submitted to queue <short>.
Job <8241115> is submitted to queue <short>.
Job <8241118> is submitted to queue <short>.
Job <8241121> is submitted to queue <short>.
Job <8241123> is submitted to queue <short>.
Job <8241126> is submitted to queue <short>.
Job <8241129> is submitted to queue <short>.
Job <8241132> is submitted to queue <short>.
Job <8241135> is submitted to queue <short>.
Job <8241138> is submitted to queue <short>.
Job <8241144> is submitted to queue <short>.
Job <8241147> is submitted to queue <short>.
Job <8241150> is submitted to queue <short>.
Job <8241153> is submitted to queue <short>.
Job <8241156> is submitted to queue <short>.
Job <8241159> is submitted to queue <short>.
Job <8241162> is submitted to queue <short>.
Job <8241165> is submitted to queue <short>.
Job <8241168> is submitted to queue <short>.
Job <8241171> is submitted to queue <short>.
Job <8241174> is submitted to queue <short>.
Job <8241177> is submitted to queue <short>.
Job <82411

Job <8241606> is submitted to queue <short>.
Job <8241609> is submitted to queue <short>.
Job <8241612> is submitted to queue <short>.
Job <8241615> is submitted to queue <short>.
Job <8241618> is submitted to queue <short>.
Job <8241621> is submitted to queue <short>.
Job <8241624> is submitted to queue <short>.
Job <8241627> is submitted to queue <short>.
Job <8241630> is submitted to queue <short>.
Job <8241633> is submitted to queue <short>.
Job <8241636> is submitted to queue <short>.
Job <8241639> is submitted to queue <short>.
Job <8241642> is submitted to queue <short>.
Job <8241645> is submitted to queue <short>.
Job <8241648> is submitted to queue <short>.
Job <8241651> is submitted to queue <short>.
Job <8241654> is submitted to queue <short>.
Job <8241656> is submitted to queue <short>.
Job <8241660> is submitted to queue <short>.
Job <8241662> is submitted to queue <short>.
Job <8241665> is submitted to queue <short>.
Job <8241668> is submitted to queue <short>.
Job <82416